In [3]:
############################################################################
#
# WHEN : APRIL 14,2020
#
# WHAT : A) FLUX&TIME-AUTOMATE-ONE.ipynb
#
# HOW : You have a choice here in this notebook:
#           1) Run it through each OBSID one by one
#                             or
#           2) Run it en masse - this is just a few cells below *USE THIS !
#
#       What this notebook does is this :
#           1) It basically looks at the flux for you :
#           & 
#           2) FLUX IS CALCULATED AT 40'' only for ALL 
#
#
# INPUT : You must have already chandra repro that Obsid before you try this
#               str(Obsid+'/repro/acisf'+ ObsidZero+'_repro_evt2.fits')
#               (MANUAL)
#         You must also give it the OBSID that you want to look at 
#               ObsidLIST (MANUAL, this is only if you iterate thru all)
#
# OUTPUT : You get the region defined within 40 arc-seconds
#               str(Obsid+"_fk.reg")  -- in fk units
#               str(Obsid+".reg")     -- in physical units
#               str(Obsid+"_bkg.reg") -- this is an annulus
#          &
#          This is the flux file in general
#               SRCFLUX = str(Obsid+'_srcflux_broad.flux') 
#          &
#          It prints out the following per OBSID for you  
#               NET_COUNTS
#               NET_ERR
#               NET_RATE 		DECIDE MANUALLY IF THIS IS HIGH OR LOW FLUX
#               ERR_RATE
#          &
#          After you have decided which is high and low flux, put it in this
#          order into a list: [high,low,high,low....]
#               LIST (MANUAL)
#
############################################################################

In [4]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import Angle
from astropy import wcs
from astropy.io import fits
import math
from Library import *
import os
import sys
import re
plt.rcParams['figure.figsize']=(18,18)
%matplotlib inline
print(os.getcwd())

from astropy import units as u
from astropy.coordinates import SkyCoord

/Users/tzegoh/Desktop/ciao


In [5]:
# Sherpa specific 
from sherpa_contrib.all import *
from sherpa import *
from ciao_contrib.runtool import *

from ciao_contrib.runtool import acis_process_events, dmstat, dmmerge
import ciao_contrib.runtool
import ciao_contrib.runtool as rt
import sherpa.astro.ui as ui
from sherpa_contrib.profiles import *
from six.moves.configparser import *
from sherpa.astro.ui import *
# Chips
#import chips_contrib.utils
#from chips_contrib.utils import *
#from chips_contrib import *
#from pychips import *
#from pychips.hlui import *
#from pycrates import *

# Marx
from sherpa_contrib.marx import *
# ChaRT
from sherpa_contrib.chart import *

# DS9
import subprocess
import glob
from glob import glob

from paramio import *
import paramio

#Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

In [6]:
def AddZerosToObsid(Obsid):
    ''' OK, this will turn add zeros in front of the Observation ID.'''
    if len(Obsid)==1:
        Obsid = str('0000'+Obsid)
    elif len(Obsid)==2:
        Obsid = str('000'+Obsid) 
    elif len(Obsid)==3:
        Obsid = str('00'+Obsid)
    elif len(Obsid)==4:
        Obsid = str('0'+Obsid)
    return Obsid

def GiveMeRaAndDec(fkName):
    '''This will give you the RA and Dec. This is in units of 
     Right Ascension:  hours ;  minutes ;  seconds
     Declination:  degrees ; arcminutes ; arcseconds'''
    f= open(fkName,"r")
    for lines in f:
        print(lines)
        linesSplit = re.split('[,()]',lines)
        print(linesSplit,'linesSplit')
        RA,DEC = linesSplit[1],linesSplit[2]
        break # you only want the first one
    f.close()
    return(RA,DEC)

In [ ]:
# THE FOLLOWING IS FOR ONE-BY-ONE OBSID INPUT
# FOR AUTOMATED INPUT, SCROLL ON DOWN 

In [120]:
Obsid = str(input("Enter the Obsid that you want : ") )

Enter the Obsid that you want : 2186


In [121]:
# We name it very early on, so that there is no confusion 
ObsidZero = AddZerosToObsid(Obsid)

In [122]:
# EXAMPLE ds9 1595/repro/acisf01595_repro_evt2.fits
runThisFile = str(Obsid + '/repro/acisf'+ObsidZero+'_repro_evt2.fits')
print(runThisFile)

2186/repro/acisf02186_repro_evt2.fits


In [123]:
OriginalRepro = str(Obsid+'/repro/acisf'+ ObsidZero+'_repro_evt2.fits')
print(OriginalRepro)

2186/repro/acisf02186_repro_evt2.fits


In [124]:
' Keep it in fk 9 units for ra and dec, and then arcseconds for size of the radius '

RA = dmkeypar(OriginalRepro, 'RA_TARG', echo='yes' )
print(RA,'RA')

DEC = dmkeypar(OriginalRepro, 'DEC_TARG', echo='yes' )
if float(DEC) > 0 :
    DEC = str('+'+DEC)
print(DEC,'DEC')


# You got to get at least the right units !!!
# This is an approximate centre 
# We need this initial approximation before we can evntually get to the true brightest pixel
c = SkyCoord(ra=float(RA)*u.degree, dec=float(DEC)*u.degree)
HMSDMS = c.to_string('hmsdms',sep=":")
Coords = HMSDMS.split()
RA_apx = Coords[0]
DEC_apx = Coords[1]
print(RA_apx)
print(DEC_apx)

# OK, here you set the radius that you want to look at. 
# For the flux, since it will take a longtime to look 
# You MUST do this in arcseconds 
radius = 40
radius_arcs= str(str(radius)+'\'\'')
print(radius_arcs)

275.48875 RA
+64.343444 DEC
18:21:57.3
+64:20:36.3984
40''


In [125]:
# OK, now we are just creating an event file centered on the RA & DEC
# and putting 40 arcsecond radius around it 

GetBrightestPix=dmcopy(infile = runThisFile+"[sky=circle("+RA_apx+","+DEC_apx+","+radius_arcs+")]",
       outfile='GetBrightestPix',clobber='True')
print(GetBrightestPix)


None


In [126]:
# Honestly, all we want is just the ra and dec in the physical coordiantes
# This is just the coordinate system in each pixel 
# This gives the brightest pixel ie the real centre 
dmstat('GetBrightestPix'+"[bin sky=1][cols ra,dec]")
maxloc=dmstat.out_max_loc
 
ra1 = maxloc.split(',')[0]
dec1 =maxloc.split(',')[1]

ra = str(round(float(ra1)))
dec= str(round(float(dec1)))


# This is the ra and dec of the brightest pixel in the physical pixel coords
print(ra)
print(dec)
radius_pix = str(radius/0.492) # units in pixel
print(radius_pix)
annulus_pix = str((radius+10)/0.492) # units in pixel for outer annulus
print(annulus_pix)
 

# This is the coordinateds of the brightest pixel in terms of the physical pixel coords
dmcoords.punlearn()
dmcoords(runThisFile,asol='none',option='sky',x=ra,y=dec)

ra_FK = dmcoords.ra
dec_FK = dmcoords.dec
# The radius is now expressed in minutes 
radius_FK = str(str(float(radius/60)) + '\'')
            

print(ra_FK)
print(dec_FK)
print(radius_FK)



3914
4206
81.30081300813008
101.6260162601626
18:21:57.236
+64:20:36.20
0.6666666666666666'


In [127]:
# OK, now we are just creating an event file centered on the ra & dec of the brightest pixel
# and putting 5 arcsecond radius around it 
TheBrightestPix=dmcopy(infile = runThisFile+"[sky=circle("+ra+","+dec+",40'')]",
       outfile='TheBrightestPix',clobber='True')
print(TheBrightestPix)

# If you need to , check it against the pix that is you can open up 
# OK, this is actually the event file with a 40 arcsecond 
# It's automated now so that you don't even have to open this at all 

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#subprocess.Popen(["ds9", 'TheBrightestPix'])
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

None


In [128]:
# Creating the fk file 
filename = str(Obsid+"_fk.reg")
f= open(filename,"w+")
words = str('circle('+  ra_FK +','+ dec_FK +','+ radius_FK +')' )
print(words)
f.write(words)
f.close()

# Creating the reg file 
filename = str(Obsid+".reg")
f= open(filename,"w+")
words = str('circle('+  ra +','+ dec +','+ radius_pix +')' )
print(words)
f.write(words)
f.close()

# Creating the bkg_reg file 
filename = str(Obsid+"_bkg.reg")
f= open(filename,"w+")
words = str('annulus('+  ra +','+ dec +','+ radius_pix +','+annulus_pix +')' )
print(words)
f.write(words)
f.close()

print('yes')

circle(18:21:57.236,+64:20:36.20,0.6666666666666666')
circle(3914,4206,81.30081300813008)
annulus(3914,4206,81.30081300813008,101.6260162601626)
yes


In [129]:
# NOW, YOU GET THE PHYSICAL + BACKGROUND FILE

# REF: This was how we did it in the past. But now we can automate it fast 
#subprocess.Popen(["ds9", OriginalRepro])

'''1) FK
# AND ALSO SAVE THE FK COORDINATES :
# str(Obsid +"_fk.reg") as a region file USING ONE CIRCLE
# Radius : 40 physical pixel, Save as Ciao format, fk5 Coordinate Systems


2) You also have to save the region + background region
    str(Obsid +".reg") + str(Obsid +"_bkg.reg")
    Do not bother blocking out anything at this point
        Just observe if you need to block out the jets or not 
    
    For foreground, just save it as a circle of radius 40
    For background, just save at annuli 40 to 50
    
    
    or
    
    If you are doing is 500 physical pixel, SAVE AS :
        Radius : 500 physical pixel, Save as Ciao format, Physical Coordinate Systems
        str(Obsid +"_250.reg") + str(Obsid +"_bkg_250.reg")
    
    For foreground, just save it as a circle of radius 250
    For background, just save at annuli 250 to 260


# OK, note that it's not a good idea to save at 250 physical radius. This is too big,
and you have way too many to do one by one. It's just better to save at 40 for now,
so that you at least standardize the area of the flux. You save this fast, and then
later save a new one at 250 for the ones that you are sure are highest and lowest flux


'''

'1) FK\n# AND ALSO SAVE THE FK COORDINATES :\n# str(Obsid +"_fk.reg") as a region file USING ONE CIRCLE\n# Radius : 40 physical pixel, Save as Ciao format, fk5 Coordinate Systems\n\n\n2) You also have to save the region + background region\n    str(Obsid +".reg") + str(Obsid +"_bkg.reg")\n    Do not bother blocking out anything at this point\n        Just observe if you need to block out the jets or not \n    \n    For foreground, just save it as a circle of radius 40\n    For background, just save at annuli 40 to 50\n    \n    \n    or\n    \n    If you are doing is 500 physical pixel, SAVE AS :\n        Radius : 500 physical pixel, Save as Ciao format, Physical Coordinate Systems\n        str(Obsid +"_250.reg") + str(Obsid +"_bkg_250.reg")\n    \n    For foreground, just save it as a circle of radius 250\n    For background, just save at annuli 250 to 260\n\n\n# OK, note that it\'s not a good idea to save at 250 physical radius. This is too big,\nand you have way too many to do one b

In [130]:
# REF ONLY
#things = dmstat(runThisFile+"[cols ra,dec]")
#print(things)


In [131]:
fkName = str(Obsid+'_fk.reg')
print(fkName)
RA,DEC = GiveMeRaAndDec(fkName)
print('RA:',RA,'DEC:',DEC)


2186_fk.reg
circle(18:21:57.236,+64:20:36.20,0.6666666666666666')
['circle', '18:21:57.236', '+64:20:36.20', "0.6666666666666666'", ''] linesSplit
RA: 18:21:57.236 DEC: +64:20:36.20


In [132]:
#RADIUS = str(input("What is the physical pixel radius of the region : ") )
RADIUS = str(radius) # fudicial is 40

In [133]:
#srcflux 1595/repro/acisf01595_repro_evt2.fits "0:20:25.2231 +15:40:54.676" clobber='yes'

srcflux.punlearn()

if int(RADIUS) == 40: # our fudicial radius 
    OUTROOT = str(Obsid + '_srcflux')
    SRCREG = str("region("+Obsid+".reg)")
    BKGREF = str("region("+Obsid+"_bkg.reg)")
    
else:
    OUTROOT = str(Obsid + '_srcflux_'+RADIUS)
    SRCREG = str("region("+Obsid+"_"+RADIUS+".reg)")
    BKGREF = str("region("+Obsid+"_bkg_"+RADIUS+".reg)")


print(OUTROOT)    
print(SRCREG) 
print(BKGREF) 
    
POS = str(RA + ' '+ DEC)


ReadOut = srcflux(infile= OriginalRepro,
                  outroot = OUTROOT, 
                  pos = POS,
                  srcreg=SRCREG,
                  bkgreg=BKGREF,
                  clobber='True'
                     )
print(ReadOut)

2186_srcflux
region(2186.reg)
region(2186_bkg.reg)
srcflux
          infile = 2186/repro/acisf02186_repro_evt2.fits
             pos = 18:21:57.236 +64:20:36.20
         outroot = 2186_srcflux
           bands = broad
          srcreg = region(2186.reg)
          bkgreg = region(2186_bkg.reg)
         bkgresp = yes
       psfmethod = ideal
         psffile = 
            conf = 0.9
         binsize = 1
         rmffile = 
         arffile = 
           model = xspowerlaw.pow1
       paramvals = pow1.PhoIndex=2.0
        absmodel = xsphabs.abs1
       absparams = abs1.nH=%GAL%
           abund = angr
         fovfile = 
        asolfile = 
         mskfile = 
        bpixfile = 
         dtffile = 
         ecffile = CALDB
        parallel = yes
           nproc = INDEF
          tmpdir = /tmp
         clobber = yes
         verbose = 1
            mode = hl

Processing OBI 000
Extracting counts
Setting Ideal PSF : alpha=1 , beta=0
Getting net rate and confidence limits
Getting model in

In [134]:
#dmlist "1595_srcflux_broad.flux[col BG_RATE,COUNT_RATE,COUNT_RATE_ERR,\
#BG_RATE,NET_COUNTS,NET_ERR,NET_RATE,ERR_RATE]" data,clean
if int(RADIUS) == 40: # our fudicial radius 
    SRCFLUX = str(Obsid+'_srcflux_broad.flux')

else:
    SRCFLUX = str(Obsid+'_srcflux_'+RADIUS+'_broad.flux')
    
cols = dmlist(SRCFLUX, opt='cols,data,clean' )
print(cols)


 
--------------------------------------------------------------------------------
Columns for Table Block HISTOGRAM
--------------------------------------------------------------------------------
 
ColNo  Name                 Unit        Type             Range
   1   sky(X,Y)             pixel        Real8          -Inf:+Inf            Position
   2   EQPOS(RA,Dec)        deg          Real8          -360.0:      360.0   Position
   3   SHAPE                             String[16]                          Region shape type
   4   R[2]                 pixel        Real8(2)       -Inf:+Inf            Radius
   5   RMID                 pixel        Real8          -Inf:+Inf            Mean of valid R[2] values
   6   ROTANG[2]            pixel        Real8(2)       -Inf:+Inf            Angle
   7   COMPONENT                         Int2           -                    Component number
   8   COUNTS               count        Real8          -Inf:+Inf            Counts
   9   ERR_COUNTS     

In [135]:
#dmlist "1595_srcflux_broad.flux[col BG_RATE,COUNT_RATE,COUNT_RATE_ERR,\
#BG_RATE,NET_COUNTS,NET_ERR,NET_RATE,ERR_RATE]" data,clean
NET_COUNTS= dmkeypar(SRCFLUX,'NET_COUNTS',echo='yes')
print(NET_COUNTS,'NET_COUNTS')
NET_ERR= dmkeypar(SRCFLUX,'NET_ERR',echo='yes')
print(NET_ERR,'NET_ERR')

NET_RATE= dmkeypar(SRCFLUX,'NET_RATE',echo='yes')# THIS IS THE ONE TO LOOK AT !
print(NET_RATE,'NET_RATE')
ERR_RATE= dmkeypar(SRCFLUX,'ERR_RATE',echo='yes')
print(ERR_RATE,'ERR_RATE')


28668.0761903743 NET_COUNTS
245.105395773357 NET_ERR
0.17369553521187 NET_RATE
0.00148505650045905 ERR_RATE


In [136]:
''' This is it. YOU ARE DONE!!!!! ANYTHING BELOW IS FOR R.E.F ONLY'''

' This is it. YOU ARE DONE!!!!! ANYTHING BELOW IS FOR R.E.F ONLY'

In [ ]:
####### SSSS TTTTT OOOO PPPPPP ##########################

In [ ]:
#S

In [ ]:
#T

In [ ]:
#O

In [ ]:
#P

In [ ]:
#!

In [ ]:
######## THIS BELOW IS FOR AUTOMATING THE ABOVE !!!!!!!!

In [ ]:
ObsidLIST = (2310,2311,2418,362,12076,12077,12078,12079,12080,12081,12082,12083,\
               12084,12085,12086,12087,6831,6832,6833,6834,6835,6836,6837,6838,6839,\
               6840,11534,11535,11536,11537,11538,11539,12831,12832,13191,13195,13960,\
               13961,14513,14514,14515,14516,14517,14518,16316,16317,19638,19639,19640,\
               19641,2241,3957,3158,12794,14212,14213,14214,20603,1662,5244,4089,819,\
               3436,907,5776,2209,2130,5627,2179,2126,377)

In [7]:
ObsidLIST =[9398,9845,9846,9848]

In [9]:
# OK, so you want to automate it 
''' You are choosing the flux for the highest and lowest flux 
in a region of 40 arcseconds around the brightest pixel.'''

for i in ObsidLIST:
    Obsid = str(i)
    print(Obsid)

    # We name it very early on, so that there is no confusion 
    ObsidZero = AddZerosToObsid(Obsid)

    # EXAMPLE ds9 1595/repro/acisf01595_repro_evt2.fits
    runThisFile = str(Obsid + '/repro/acisf'+ObsidZero+'_repro_evt2.fits')
    print(runThisFile)
    OriginalRepro = str(Obsid+'/repro/acisf'+ ObsidZero+'_repro_evt2.fits')
    print(OriginalRepro)

    ' Keep it in fk 9 units for ra and dec, and then arcseconds for size of the radius '
    RA = dmkeypar(OriginalRepro, 'RA_TARG', echo='yes' )
    print(RA,'RA')

    DEC = dmkeypar(OriginalRepro, 'DEC_TARG', echo='yes' )
    if float(DEC) > 0 :
        DEC = str('+'+DEC)
    print(DEC,'DEC')
    # You got to get at least the right units !!!
    # This is an approximate centre 
    # We need this initial approximation before we can evntually get to the true brightest pixel
    c = SkyCoord(ra=float(RA)*u.degree, dec=float(DEC)*u.degree)
    HMSDMS = c.to_string('hmsdms',sep=":")
    Coords = HMSDMS.split()
    RA_apx = Coords[0]
    DEC_apx = Coords[1]
    #print(RA_apx)
    #print(DEC_apx)
    # OK, here you set the radius that you want to look at. 
    # For the flux, since it will take a longtime to look 
    # You MUST do this in arcseconds 
    radius = 40
    radius_arcs= str(str(radius)+'\'\'')
    #print(radius_arcs)

    # OK, now we are just creating an event file centered on the RA & DEC
    # and putting 40 arcsecond radius around it 

    GetBrightestPix=dmcopy(infile = runThisFile+"[sky=circle("+RA_apx+","+DEC_apx+","+radius_arcs+")]",
           outfile='GetBrightestPix',clobber='True')
    print(GetBrightestPix)
    # Honestly, all we want is just the ra and dec in the physical coordiantes
    # This is just the coordinate system in each pixel 
    # This gives the brightest pixel ie the real centre 
    dmstat('GetBrightestPix'+"[bin sky=1][cols ra,dec]")
    maxloc=dmstat.out_max_loc
    ra1 = maxloc.split(',')[0]
    dec1 =maxloc.split(',')[1]
    ra = str(round(float(ra1)))
    dec= str(round(float(dec1)))


    # This is the ra and dec of the brightest pixel in the physical pixel coords
    print(ra)
    print(dec)
    radius_pix = str(radius/0.492) # units in pixel
    #print(radius_pix)
    annulus_pix = str((radius+10)/0.492) # units in pixel for outer annulus
    #print(annulus_pix)


    # This is the coordinateds of the brightest pixel in terms of the physical pixel coords
    dmcoords.punlearn()
    dmcoords(runThisFile,asol='none',option='sky',x=ra,y=dec)
    ra_FK = dmcoords.ra
    dec_FK = dmcoords.dec
    # The radius is now expressed in minutes 
    radius_FK = str(str(float(radius/60)) + '\'')
    print(ra_FK)
    print(dec_FK)
    print(radius_FK)

    # OK, now we are just creating an event file centered on the ra & dec of the brightest pixel
    # and putting 40 arcsecond radius around it 
    TheBrightestPix=dmcopy(infile = runThisFile+"[sky=circle("+ra+","+dec+",40'')]",
           outfile='TheBrightestPix',clobber='True')
    print(TheBrightestPix)
    # If you need to , check it against the pix that is you can open up 
    # OK, this is actually the event file with a 40 arcsecond 
    # It's automated now so that you don't even have to open this at all 

    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #subprocess.Popen(["ds9", 'TheBrightestPix'])
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    # Creating the fk file 
    filename = str(Obsid+"_fk.reg")
    f= open(filename,"w+")
    words = str('circle('+  ra_FK +','+ dec_FK +','+ radius_FK +')' )
    #print(words)
    f.write(words)
    f.close()
    # Creating the reg file 
    filename = str(Obsid+".reg")
    f= open(filename,"w+")
    words = str('circle('+  ra +','+ dec +','+ radius_pix +')' )
    #print(words)
    f.write(words)
    f.close()
    # Creating the bkg_reg file 
    filename = str(Obsid+"_bkg.reg")
    f= open(filename,"w+")
    words = str('annulus('+  ra +','+ dec +','+ radius_pix +','+annulus_pix +')' )
    #print(words)
    f.write(words)
    f.close()

    # Get the FK name 
    fkName = str(Obsid+'_fk.reg')
    #print(fkName)
    RA,DEC = GiveMeRaAndDec(fkName)
    #print('RA:',RA,'DEC:',DEC)
    #srcflux 1595/repro/acisf01595_repro_evt2.fits "0:20:25.2231 +15:40:54.676" clobber='yes'

    srcflux.punlearn()
    
    # This is set as 40; really at this point, we won't keep to the fiducial radius 
    RADIUS = str(radius)

    if int(RADIUS) == 40: # our fudicial radius 
        OUTROOT = str(Obsid + '_srcflux')
        SRCREG = str("region("+Obsid+".reg)")
        BKGREF = str("region("+Obsid+"_bkg.reg)")

    else:
        OUTROOT = str(Obsid + '_srcflux_'+RADIUS)
        SRCREG = str("region("+Obsid+"_"+RADIUS+".reg)")
        BKGREF = str("region("+Obsid+"_bkg_"+RADIUS+".reg)")
    #print(OUTROOT)    
    #print(SRCREG) 
    #print(BKGREF)  
    POS = str(RA + ' '+ DEC)
    ReadOut = srcflux(infile= OriginalRepro,
                      outroot = OUTROOT, 
                      pos = POS,
                      srcreg=SRCREG,
                      bkgreg=BKGREF,
                      clobber='True'
                         )
    #print(ReadOut)

    #dmlist "1595_srcflux_broad.flux[col BG_RATE,COUNT_RATE,COUNT_RATE_ERR,\
    #BG_RATE,NET_COUNTS,NET_ERR,NET_RATE,ERR_RATE]" data,clean
    if int(RADIUS) == 40: # our fudicial radius 
        SRCFLUX = str(Obsid+'_srcflux_broad.flux')

    else:
        SRCFLUX = str(Obsid+'_srcflux_'+RADIUS+'_broad.flux') 
    cols = dmlist(SRCFLUX, opt='cols,data,clean' )
    #print(cols)

    #dmlist "1595_srcflux_broad.flux[col BG_RATE,COUNT_RATE,COUNT_RATE_ERR,\
    #BG_RATE,NET_COUNTS,NET_ERR,NET_RATE,ERR_RATE]" data,clean
    NET_COUNTS= dmkeypar(SRCFLUX,'NET_COUNTS',echo='yes')
    print(NET_COUNTS,'NET_COUNTS')
    NET_ERR= dmkeypar(SRCFLUX,'NET_ERR',echo='yes')
    print(NET_ERR,'NET_ERR')

    NET_RATE= dmkeypar(SRCFLUX,'NET_RATE',echo='yes')
    print(NET_RATE,'NET_RATE')
    ERR_RATE= dmkeypar(SRCFLUX,'ERR_RATE',echo='yes')
    print(ERR_RATE,'ERR_RATE')

    #Write it into the file !!
    filename = str("HighFluxAndLowFlux.txt")
    f= open(filename,"a+")

    #Observation ID 
    words = str(str(Obsid)+'\n')
    f.write(words)
    # Radius
    words = str(str(radius)+'_physical radius in arc seconds \n')
    f.write(words)

    # The flux 
    words = str(str(NET_COUNTS)+'\n')
    f.write(words)
    words = str(str(NET_ERR)+'\n')
    f.write(words)
    words = str(str(NET_RATE)+'\n') # THIS IS THE ONE TO LOOK AT !
    f.write(words)
    words = str(str(ERR_RATE)+'\n'+'\n')
    f.write(words)

    f.close()
    

9398
9398/repro/acisf09398_repro_evt2.fits
9398/repro/acisf09398_repro_evt2.fits
275.489167 RA
+64.343611 DEC
None
4045
3952
18:21:57.204
+64:20:36.08
0.6666666666666666'
None
circle(18:21:57.204,+64:20:36.08,0.6666666666666666')
['circle', '18:21:57.204', '+64:20:36.08', "0.6666666666666666'", ''] linesSplit
21190.3980757088 NET_COUNTS
201.061392095801 NET_ERR
0.628223515360779 NET_RATE
0.00596078913168464 ERR_RATE
9845
9845/repro/acisf09845_repro_evt2.fits
9845/repro/acisf09845_repro_evt2.fits
275.489167 RA
+64.343611 DEC
None
4045
3953
18:21:57.202
+64:20:36.54
0.6666666666666666'
None
circle(18:21:57.202,+64:20:36.54,0.6666666666666666')
['circle', '18:21:57.202', '+64:20:36.54', "0.6666666666666666'", ''] linesSplit
15033.7751961215 NET_COUNTS
170.060909345807 NET_ERR
0.621353516471856 NET_RATE
0.00702870321379291 ERR_RATE
9846
9846/repro/acisf09846_repro_evt2.fits
9846/repro/acisf09846_repro_evt2.fits
275.489167 RA
+64.343611 DEC
None
4045
3952
18:21:57.196
+64:20:36.07
0.6666666

In [6]:
# REF ONLY FROM THIS POINT ON 


# This is old yeah. This is when you use look for the flux without specifying region.
dataLeft = np.load('HighAndLowFluxArray.npy',allow_pickle=True)
print(dataLeft[:,[0,1,7,9]])

[['CXOMP_J111816.8+074557' '7757' 0.00114634372394273 2]
 ['CXOMP_J111816.8+074557' '10796' 0.0150460267819783 1]
 ['CXOMP_J142438.0+225600' '367' 0.0871253578297048 1]
 ['CXOMP_J142438.0+225600' '3330' 0.0 2]
 ['CXOMP_J112740.1+565014' '4659' 0.00066060543846677 1]
 ['CXOMP_J112740.1+565014' '4660' -0.000111311478280046 2]
 ['CXOMP J002025.2+154054' '1595' -0.000655299651040588 2]
 ['CXOMP J002025.2+154054' '10234' 0.0244954880327842 1]
 ['3c093' '16051' 0.061113934297135 1]
 ['CXOMPJ021043.2-001817' '2081' 0.00102259080554867 2]
 ['CXOMPJ021043.2-001817' '6820' 0.00687616542546761 1]
 ['CXOMPJ151023.9+073844' '13886' 0.00130783746457103 1]
 ['CXOMPJ151023.9+073844' '18225' -0.000131526674370658 2]
 ['PG1634+706' '62' 0.223756009219682 1]
 ['PG1634+706' '1269' 0.12849268606729 2]
 ['PDS 456' '17247' -3.37197604098957e-05 2]
 ['PDS 456' '17452' 0.0618782205467229 1]
 ['KUV\xa018217+6419' '1599' 0.130449436947531 1]
 ['KUV\xa018217+6419' '2418' 0.102191730393946 2]
 ['MRK 509' '11387' 0

In [7]:
print(dataLeft[:,1])

['7757' '10796' '367' '3330' '4659' '4660' '1595' '10234' '16051' '2081'
 '6820' '13886' '18225' '62' '1269' '17247' '17452' '1599' '2418' '11387'
 '13864' '16539' '16540' '9151' '9899' '20124' '20887' '362' '12084'
 '6839' '12831' '2241' '3957' '3158' '12794' '1662' '5244' '4089' '819'
 '3013' '3436' '907' '5776' '2209' '2130' '5627' '2179' '2126']


In [17]:
Header = dmlist(OriginalRepro, opt='header')
print(Header)



 
--------------------------------------------------------------------------------
Header keys for block EVENTS
--------------------------------------------------------------------------------
 
 --  COMMENT                                            +------------------+
 --  COMMENT                                            | AXAF FITS   File |
 --  COMMENT                                            +------------------+
 --  COMMENT               *********************************************************
 --  COMMENT               >    This file is written following certain AXAF-ASC    <
 --  COMMENT               >    conventions which are documented in ASC-FITS-2.0   <
 --  COMMENT               *********************************************************
 --  COMMENT                                     / Configuration control block--------------------
0001 ASOLFILE             pcadf076358603N004_asol1.fits  String       
0002 THRFILE              acisD1996-11-01evtspltN0002.fits Strin

In [18]:
'''Here it is : the time from the repro file
This is just for REF. You won't be using this. You would be finding
the RA and DEC of the central of the point source by actually looking at the reg file.
'''

LiveTime = dmkeypar(OriginalRepro, 'LIVETIME', echo='yes' )
print(LiveTime,'LiveTime')

RA = dmkeypar(OriginalRepro, 'RA_NOM', echo='yes' )
print(RA,'RA')

DEC = dmkeypar(OriginalRepro, 'DEC_NOM', echo='yes' )
print(DEC,'DEC')

26492.452845296 LiveTime
169.56412340468 RA
7.7551196193489 DEC


In [ ]:
''' YOU CAN USE THE BELOW FOR CHECKING THE RESULTS IN BASH R.E.F. ONLY '''

In [71]:
# WITH REGION 

punlearn srcflux
pset srcflux srcreg="region(1595.reg)"
pset srcflux bkgreg="region(1595_bkg.reg)"
srcflux 1595/repro/acisf01595_repro_evt2.fits "0:20:25.2836 +15:40:54.549" clobber='yes'


      Position                               0.5 - 7.0 keV
                                             Value        90% Conf Interval
#0001|0 20 25.28 +15 40 54.5  Rate           0.0414 c/s (0.0388,0.0441)
                              Flux           2.94E-13 erg/cm2/s (2.75E-13,3.12E-13)
                              Mod.Flux       2.25E-13 erg/cm2/s (2.1E-13,2.39E-13)
                              Unabs Mod.Flux 2.41E-13 erg/cm2/s (2.26E-13,2.57E-13)


SyntaxError: invalid syntax (<ipython-input-71-470c048a23b8>, line 1)

In [ ]:
dmlist "1595_srcflux2_broad.flux[col counts,ERR_COUNTS, bg_counts, BG_ERR,exposure]" data,clean
#  COUNTS               ERR_COUNTS           BG_COUNTS            BG_ERR               EXPOSURE
                900.0                 30.0                 43.0         6.5574385243     19884.5415938902

In [ ]:
dmlist "1595_srcflux2_broad.flux[col BG_RATE,COUNT_RATE,COUNT_RATE_ERR,\
BG_RATE,NET_COUNTS,NET_ERR,NET_RATE,ERR_RATE]" data,clean

#  BG_RATE              COUNT_RATE           COUNT_RATE_ERR       NET_COUNTS           NET_ERR              NET_RATE             ERR_RATE
     0.00216248384691     0.04526128981905     0.00150870966063       823.6327220504        32.1811591353     0.04142075481908     0.00161840085593

In [ ]:
#Lin for the photon flux
4.00E-02



In [ ]:
RA,DEC = ('11:18:15.3896 +07:45:18.4306')

In [ ]:
punlearn srcflux

srcflux 363/repro/acisf00363_repro_evt2.fits '11:18:15.3896 +07:45:18.4306' clobber='yes'


